In [ ]:
import geopandas
import pandas as pd
import numpy as np
import os
import osmnx as ox
import matplotlib.pyplot as plt
import shapely as shp
import xlrd, xlwt
from geopy import Nominatim
from IPython.display import Image
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

## read EPCs files and Archetypes

In [ ]:
file = pd.read_csv('500236_Statistik_Standarduttag_Forskning/efter_v260/data/std_uttag_efter_260.csv',sep=',', decimal=',')
file1 = pd.read_csv('500236_Statistik_Standarduttag_Forskning/fore_v260/data/std_uttag_fore_260.csv',sep=',', decimal=',')
file = pd.concat([file1, file], ignore_index=True)
Arche = pd.read_excel('BESTI_building type.xlsx')

In [ ]:
# nfile = pd.DataFrame(file.loc[:,'IdKommun'][file.loc[:,'EgenByggnadsKat']=='Flerbostadshus' and file.loc[:,'EgenNybyggAr'] <= 2005])
nfile = file[(file['EgenByggnadsKat']=='Flerbostadshus') & (file['EgenNybyggAr']<=2005)]
nfile_DH = nfile[nfile['EgiFjarrvarme'].notnull()]

## Stockholm

In [ ]:
sth = nfile_DH[nfile_DH['IdKommun']=='Stockholm']
sth = sth.drop_duplicates(subset='IdAdr', keep='first')

In [ ]:
sth.to_excel('C:\OneDrive\Master_Thesis\data_and_results\Building_info\stockholm.xlsx')

## malmo_DH

In [ ]:
malmo = nfile_DH[nfile_DH['IdKommun']=='Malmö']
malmo = malmo.drop_duplicates(subset='IdAdr', keep='first')

In [ ]:
malmo.to_excel('C:\OneDrive\Master_Thesis\data_and_results\Building_info\malmo_DH.xlsx')

In [ ]:
malmo1929 = malmo[malmo['IdAdr']=='Spånehusvägen 46A']
malmo1971 = malmo[malmo['IdAdr']=='Kantatgatan 60']
malmo1979 = malmo[malmo['IdAdr']=='Rolfsgatan 13']
malmo1990 = malmo[malmo['IdAdr']=='Skansgatan 1']
malmo2000 = malmo[malmo['IdAdr']=='Rodergatan 8A']
malmo_DH = pd.concat([malmo1929, malmo1971, malmo1979, malmo1990, malmo2000])
malmo_DH

## Lomma_86_95

In [ ]:
# hels = nfile[nfile['IdKommun']=='Helsingborg']
hoj_lomma = nfile[(nfile['IdAdr'].str.contains('Höjeågatan|Sandhamnsgatan')) & (nfile['IdKommun'] == 'Lomma')]
hoj_lomma = hoj_lomma[hoj_lomma['IdRapportLM'].isnull()]

# select several buildings for simplify and example
# hoj_lomma = hoj_lomma[(hoj_lomma['IdAdr'].str.contains('273|365|359|405|389|387|427|425|421|419|351|263|251|259|219|211|190|224|229|241|246|120|130|150|170|89|77|65|53|41|25'))]
hoj_lomma = hoj_lomma[(hoj_lomma['IdAdr'] == 'Höjeågatan 273') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 365') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 359') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 405') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 389') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 387') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 427') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 425') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 421') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 419') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 351') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 263') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 251') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 259') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 219') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 211') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 191') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 225') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 241') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 245') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 119') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 131') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 151') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 171') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 89') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 77') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 65') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 53') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 41') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 25') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 3') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 9') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 15') |
                     (hoj_lomma['IdAdr'] == 'Höjeågatan 21') |
                     (hoj_lomma['IdAdr'] == 'Sandhamnsgatan 9')]

hoj_lomma.drop_duplicates(subset=['IdAdr'], keep='first', inplace=True)

In [ ]:
hoj_lomma.head()

## Ostersund DH

In [ ]:
ostersund = nfile_DH[nfile_DH['IdLan']=='Jämtlands län']

In [ ]:
ostersund1956 = ostersund[ostersund['IdAdr']=='Krondikesvägen 36A']
ostersund1979 = ostersund[ostersund['IdAdr']=='Löjtnantsvägen 72']
ostersund1991 = ostersund[ostersund['IdAdr']=='Havrevägen 12']

In [ ]:
ostersund_DH = pd.concat([ostersund1956, ostersund1979, ostersund1991])

## Hammarö -60 DH

In [ ]:
hammaro_DH = nfile[(nfile['IdKommun'] == 'Hammarö')]
hammaro_DH = hammaro_DH[(hammaro_DH['IdAdr']=='Vändgatan 3') |
                  (hammaro_DH['IdAdr']=='Vändgatan 1B') |
                  (hammaro_DH['IdAdr']=='Vändgatan 7B') |
                  (hammaro_DH['IdAdr']=='Vändgatan 11B')]
hammaro_DH = hammaro_DH.drop_duplicates(subset='IdAdr', keep='first')

## Hammarö -60

In [ ]:
hammaro = nfile[(nfile['IdKommun'] == 'Hammarö')]
hammaro = hammaro[(hammaro['IdAdr']=='Lunserudsvägen 12B') |
                  (hammaro['IdAdr']=='Prästgårdsvägen 13A') |
                  (hammaro['IdAdr']=='Mellanvägen 2B') |
                  (hammaro['IdAdr']=='Vändgatan 3') |
                  (hammaro['IdAdr']=='Vändgatan 1B') |
                  (hammaro['IdAdr']=='Vändgatan 7B') |
                  (hammaro['IdAdr']=='Vändgatan 11B')]
hammaro = hammaro.drop_duplicates(subset='IdAdr', keep='first')

In [ ]:
hammaro.head()

## Malmö -60

In [ ]:
malmo = nfile[(nfile['IdKommun'] == 'Malmö')]
malmo_60 = malmo[(malmo['IdAdr']=='Roddaregatan 20A') |
        (malmo['IdAdr']=='Birger Jarlsgatan 37A') |
        (malmo['IdAdr']=='Västra Bernadottesgatan 20')]

## Malmö 61-75

In [ ]:
malmo_61_75 = malmo[(malmo['IdAdr']=='Konsultgatan 2') |
        (malmo['IdAdr']=='Konsultgatan 5') |
        (malmo['IdAdr']=='Konsultgatan 30') |
        (malmo['IdAdr']=='Konsultgatan 21') |
        (malmo['IdAdr']=='Konsultgatan 8') |
        (malmo['IdAdr']=='Konsultgatan 11') |
        (malmo['IdAdr']=='Konsultgatan 22') |
        (malmo['IdAdr']=='Konsultgatan 27') |
        (malmo['IdAdr']=='Konsultgatan 16') |
        (malmo['IdAdr']=='Konsultgatan 35') |
        (malmo['IdAdr']=='Konsultgatan 19')]

In [ ]:
# malmo_61_75

## malmo_76_85

In [ ]:
malmo_76_85 = malmo[(malmo['IdAdr']=='Lugna Gatan 3') |
        (malmo['IdAdr']=='Lugna Gatan 17') |
        (malmo['IdAdr']=='Lugna Gatan 31') |
        (malmo['IdAdr']=='Lugna Gatan 41') |
        (malmo['IdAdr']=='Lugna Gatan 47')]

## malmo_86_95

In [ ]:
malmo_86_95 = malmo[(malmo['IdAdr']=='Gullregnsvägen 2A') |
        (malmo['IdAdr']=='Jägersrovägen 5B') |
        (malmo['IdAdr']=='Jägersrovägen 7B') |
        (malmo['IdAdr']=='Jägersrovägen 9B') |
        (malmo['IdAdr']=='Jägersrovägen 11B') |
        (malmo['IdAdr']=='Jägersrovägen 13B') |
        (malmo['IdAdr']=='Jägersrovägen 15B') |
        (malmo['IdAdr']=='Jägersrovägen 17B') |
        (malmo['IdAdr']=='Jägersrovägen 19D') |
        (malmo['IdAdr']=='Jägersrovägen 19B') |
        (malmo['IdAdr']=='Poppelgatan 18B')]
malmo_86_95 = malmo_86_95.drop_duplicates(subset='IdAdr', keep='last')

## malmo_96_05

In [ ]:
malmo_96_05 = malmo[(malmo['IdAdr']=='Rodergatan 4') |
        (malmo['IdAdr']=='Propellergatan 13')]
malmo_96_05 = malmo_96_05.drop_duplicates(subset='IdAdr', keep='first')

## Single Buildings

In [ ]:
single_building = pd.concat([ostersund[ostersund['IdAdr']=='Skylvägen 11'],
                       malmo[malmo['IdAdr']=='Mäster Palmsgatan 7A'],
                       malmo[malmo['IdAdr']=='Musikantgatan 1'],
                       sth[sth['IdAdr']=='Porkalafaret 40'],
                       sth[sth['IdAdr']=='Solberga Hagväg 10']])

single_building

## Buildings of input

In [ ]:
buildings = model_cali
buildings = buildings.drop_duplicates(subset='IdAdr', keep='first')

## Data processing 

In [ ]:
add_street = list(buildings['IdAdr'])
add_city = list(buildings['IdKommun'])
addr = []
for i in range(len(add_city)):
    addr.append('{},{}'.format(add_street[i], add_city[i]))
addr

In [ ]:
nom = Nominatim(user_agent='daxug82')

### TIME CONSUMING!!! get lat and lon

In [ ]:
tags = {"building": True}
lat = []
lon = []
for j in range(len(addr)):
    posi = nom.geocode(addr[j])
    lat.append(posi.latitude)
    lon.append(posi.longitude)  

In [ ]:
buildings['latitude'] = lat
buildings['longtitude'] = lon

In [ ]:
buildings

In [ ]:
polygon = pd.DataFrame(columns=['latitude', 'longtitude', 'polygon data'])
polygon_context = pd.DataFrame(columns=['latitude', 'longtitude', 'polygon data'])
# polygon.append({'latitude':lat[1], 'longtitude':lon[1]}, ignore_index=True)

### TIME CONSUMING!!! get building Polygon data

In [ ]:
area_OSM = 0
for i in range(len(lat)):
    point = (lat[i], lon[i])
    gdf = geopandas.GeoDataFrame()
    dist = 6
    polygon.loc[i,'latitude'] = lat[i]
    polygon.loc[i,'longtitude'] = lon[i]
    gdf = ox.geometries_from_point(point, tags, dist=dist)
    gdf = gdf[gdf.area > 5e-09]
    # while geometry.empty == True:
    #     dist += 0.5
    #     gdf = ox.geometries_from_point(point, tags, dist=dist)
    polygon.loc[i,'polygon data'] = np.nan
    if gdf.empty != True:
        gdf_proj = ox.project_gdf(gdf)
        polygon.loc[i,'polygon data'] = [list(gdf_proj.geometry.exterior[row_id].coords) for row_id in range(gdf.shape[0])][0]
    area_OSM += gdf_proj.area.values[0]
    print('{}/{}'.format(i, len(lat)-1))

### Time Consuming!!! Get context polygon data

In [ ]:
for i in range(len(lat)):
    point = (lat[i], lon[i])
    gdf_context = geopandas.GeoDataFrame()
    dist = 100
    polygon_context.loc[i,'latitude'] = lat[i]
    polygon_context.loc[i,'longtitude'] = lon[i]
    gdf_context = ox.geometries_from_point(point, tags, dist=dist)
    # while geometry.empty == True:
    #     dist += 0.5
    #     gdf = ox.geometries_from_point(point, tags, dist=dist)
    polygon_context.loc[i,'polygon data'] = np.nan
    if gdf_context.empty != True:
        gdf_context_proj = ox.project_gdf(gdf_context)
        polygon_context.loc[i,'polygon data'] = [list(gdf_context_proj.geometry.exterior[row_id].coords) for row_id in range(gdf_context.shape[0])]
    print('{}/{}'.format(i, len(lat)-1))

### Display buidling polygon

In [ ]:
# configure the inline image display
img_folder = "images"
extension = "png"
size = 240

ind = 1
point = (lat[ind], lon[ind])
print(lat[ind], lon[ind])
dist = 100
gdf = ox.geometries_from_point(point, tags, dist=dist)
if gdf.empty:
    print('empty!')
# gdf = gdf[gdf.area > 5e-09]
# gdf.to_excel(r'E:\OneDrive\Master_Thesis\Energy_Simulation\GIS Data\test.xlsx', sheet_name='1')
gdf_proj = ox.project_gdf(gdf)
bbox = ox.utils_geo.bbox_from_point(point=point, dist=50, project_utm=True)
fp = f"./{img_folder}/paris_bldgs.{extension}"
fig, ax = ox.plot_footprints(
    gdf_proj,
    bbox=bbox,
    color="w",
    filepath=fp,
    dpi=90,
    save=True,
    show=False,
    close=True,
)
Image(fp, height=size, width=size)


In [ ]:
buildings.loc[:,'polygon data'] = list(polygon['polygon data'])
buildings.loc[:, 'contex polygon data'] = list(polygon_context['polygon data'])
buildings

In [ ]:
area_EPCs = (buildings['EgenAtemp']/(buildings['EgenAntalPlan']+buildings['EgenAntalKallarplan']/2)).sum()

scale_factor = area_EPCs/area_OSM
scale_factor

buildings['EgenAtemp'].sum()
scale_factor
buildings['IdAdr']

## Predict Building Types

In [ ]:
# read files
FA = pd.read_csv('betsi_origin/FA.csv', sep=';' ,encoding='iso-8859-1', decimal=',')
FB_Klimat = pd.read_csv('betsi_origin/FB_Klimat.csv',sep=';' ,encoding='iso-8859-1', decimal=',').drop(columns=['Hustyp'])
year = pd.read_csv('betsi_origin/Byggår.csv', encoding='iso-8859-1')
FB49_YTTERVAGG = pd.read_csv('betsi_origin/FB49 Ytterv„gg.csv', sep=';', encoding='iso-8859-1', decimal=',')
FB52_YTTERVAGG = pd.read_csv('betsi_origin/FB52 Ytterv„gg.csv', sep=';', encoding='iso-8859-1', decimal=',')

# concat several features
F = pd.concat([FA, FB_Klimat],axis=1, join='inner')

# Delete unrealistic values
F = F[(F['FB52 U-medelvärde ytterväggar']>=0.1) & (F['FB87-97 U-medelvärde fönster']>0)]
F.loc[:, 'FXByggår'] = year['FXByggÃ¥r']

# select muilti-family buildings
F = F[F['Hustyp']=='F']

# building type
bt = F.loc[:,['F14ATempt', 'FA16AntalPlanOvanMark', 'FA17AntalTrappuppgangar', 'FA20Summa', 'FXByggår', 'FA11Byggnadstyp']]
bt = bt[(bt['FA11Byggnadstyp'] != 'AnnanVilken') & (bt['FA17AntalTrappuppgangar']<10)]
# bt = bt[bt['FA17AntalTrappuppgangar']<10]

le = LabelEncoder().fit(bt.iloc[:,-1])

In [ ]:
clf = joblib.load('Buidling_Types_Pre.pkl')
feature = buildings[['EgenAntalPlan', 'EgenAntalTrapphus', 'EgenAntalBolgh']]
y = clf.predict(feature)
buildings.loc[:,'Building types'] = y
buildings

## Applying Archetypes

In [ ]:
def Arche_apply(x):
    arche = []
    for j in range(len(list(x['EgenNybyggAr']))):
        i = list(x['EgenNybyggAr'])[j]
        
        if i <=1960:
            ar = 'F_60_{}'.format(list(buildings['Building types'])[j])
        elif 1960 < i <= 1975:
            ar = 'F_61_75_{}'.format(list(buildings['Building types'])[j])
        elif 1976 <= i <= 1985:
            ar = 'F_76_85_{}'.format(list(buildings['Building types'])[j])
        elif 1986 <= i <= 1995:
            ar = 'F_86_95_{}'.format(list(buildings['Building types'])[j])
        elif 1996 <= i <= 20055:
            ar = 'F_96_05_{}'.format(list(buildings['Building types'])[j])
        arche.append(ar)
    return arche

In [ ]:
arche = Arche_apply(buildings)

besti_bt = pd.read_excel('BESTI_building type.xlsx',index_col='Classification')

In [ ]:
arche_info = pd.DataFrame()
for i in range(len(arche)):
    arche_info = pd.concat([arche_info,besti_bt.loc[[arche[i]]]])

In [ ]:
buildings.loc[:, 'Archetypes'] = arche
buildings.loc[:, 'Atemp convert factor'] = scale_factor
buildings = pd.concat([buildings.reset_index(), arche_info.reset_index()], axis=1)

In [ ]:
s = list(buildings['IdKommun'])[1]
s = s.replace('ö', 'o').replace('Ö', 'o')
buildings.to_excel('Building_info/{}_{}_DH_sb.xls'.format(s, list(buildings['EgenNybyggAr'])[1]), sheet_name='sheet1')

In [ ]:
buildings.to_excel('Building_info/model calibration.xls'.format(s, list(buildings['EgenNybyggAr'])[1]), sheet_name='sheet1')

## Simulation Building Selection

In [ ]:
# # building_select = nfile[(nfile['EgenNybyggAr']<1961)&(nfile['IdKommun']=='Malmö')]

# building_select = nfile[(nfile['EgenNybyggAr']<1995) & 
#                        (nfile['EgenNybyggAr']>=1986) &
#                        (nfile['IdKommun']=='Malmö')]
# building_select.to_excel('Building info/Simu-Building selection.xls', sheet_name='1')

## xlrd test

In [ ]:
# path = 'Building info/Malmö_2001.xls'
# wb = xlrd.open_workbook(path)
# ws=wb.sheet_by_name('sheet1')

# nrows=ws.nrows                       
# ncols=ws.ncols

# A = ws.row_values(0).index('polygon data')
# polygon = []
# for i in range(1, len(ws.col_values(A))):
#     polygon.append(eval(ws.col_values(A)[i]))
# malmo_76_85['EgenAntalPlan']